# HW 4 - All About Attention

Welcome to CS 287 HW4. To begin this assignment first turn on the Python 3 and GPU backend for this Colab by clicking `Runtime > Change Runtime Type` above.  

In this homework you will be reproducing the decomposable attention model in Parikh et al. https://aclweb.org/anthology/D16-1244. (This is one of the models that inspired development of the transformer). 



## Goal

We ask that you finish the following goals in PyTorch:

1. Implement the vanilla decomposable attention model as described in that paper.
2. Implement the decomposable attention model with intra attention or another extension.
3. Visualize the attentions in the above two parts.
4. Implement a mixture of models with uniform prior and perform training with exact log marginal likelihood (see below for detailed instructions)
5. Train the mixture of models in part 4 with VAE. (This may not produce a better model, this is still a research area) 
6. Interpret which component specializes at which type of tasks using the posterior.

Consult the paper for model architecture and hyperparameters, but you are also allowed to tune the hyperparameters yourself. 

In [ ]:
!pip install -q torch torchtext opt_einsum git+https://github.com/harvardnlp/namedtensor

In [4]:
from load_data import load
from models import AttentionModel

import torch
# Text text processing library and methods for pretrained word embeddings
import torchtext
from torchtext.vocab import Vectors, GloVe

# Named Tensor wrappers
from namedtensor import ntorch, NamedTensor
from namedtensor.text import NamedField
import random
import copy

In [5]:
train_iter, val_iter, test_iter, TEXT, LABEL = load()

In [123]:
batch = next(iter(train_iter))
model = AttentionModel(TEXT, LABEL, 200, 200, 200, intra_attn = True, dropout = 0.2, device = 'cpu')

In [124]:
model(batch.premise, batch.hypothesis)

NamedTensor(
	tensor([[-0.0082, -0.0083, -0.0596,  0.0132],
        [ 0.0062,  0.0034, -0.0883, -0.0683],
        [ 0.1337,  0.0040, -0.0960,  0.1756],
        [-0.1348,  0.0497, -0.1936,  0.0818],
        [-0.0082,  0.0597, -0.0710, -0.0322],
        [-0.0954, -0.0290, -0.1964, -0.0256],
        [-0.0081,  0.0218, -0.0240,  0.0146],
        [-0.0060, -0.0100, -0.0517, -0.0161],
        [-0.0090, -0.0033, -0.1289, -0.0865],
        [-0.0742,  0.0306, -0.1139,  0.0769],
        [-0.0328,  0.0535, -0.1863, -0.1666],
        [ 0.0590,  0.1012, -0.0384,  0.0928],
        [ 0.0840, -0.0694, -0.1923,  0.0354],
        [-0.0337, -0.1111, -0.0819, -0.0135],
        [-0.0952,  0.0595, -0.0807,  0.0189],
        [ 0.0277,  0.0519,  0.0262,  0.0590]], grad_fn=<ViewBackward>),
	('batch', 'logit'))

In [125]:
model.fit(train_iter, epochs = 1, interval = 100)

batch: 0, loss: 0.013766300678253175
batch: 100, loss: 1.2436670899391173


KeyboardInterrupt: 

### Instructions for latent variable mixture model.

For the last part of this assignment we will consider a latent variable version of this model. This is a use of latent variable as a form of ensembling.

Instead of a single model, we use $K$ models $p(y | \mathbf{a}, \mathbf{b}; \theta_k)$ ($k=1,\cdots,K$), where $K$ is a hyperparameter. Let's introduce a discrete latent variable $c\sim \text{Uniform}(1,\cdots, K)$ denoting which model is being used to produce the label $y$, then the marginal likelihood is


$$
p(y|\mathbf{a}, \mathbf{b}; \theta) = \sum_{c=1}^K p(c) p(y | \mathbf{a}, \mathbf{b}; \theta_c)
$$

When $K$ is small, we can *enumerate* all possible values of $c$ to maximize the log marginal likelihood. 

We can also use variational auto encoding to perform efficient training. We first introduce an inference network $q(c| y, \mathbf{a}, \mathbf{b})$, and the ELBO is

$$
\log p(y|\mathbf{a}, \mathbf{b}; \theta)  \ge \mathbb{E}_{c \sim q(c|y, \mathbf{a}, \mathbf{b})} \log p(y|\mathbf{a},\mathbf{b}; \theta_c) - KL(q(c|y, \mathbf{a}, \mathbf{b})|| p(c)),
$$

where $p(c)$ is the prior uniform distribution. We can calculate the $KL$ term in closed form, but for the first term in ELBO, due to the discreteness of $c$, we cannot use the reparameterization trick. Instead we use REINFORCE to estimate the gradients (or see slides):

$$
\nabla \mathbb{E}_{c \sim q(c|y, \mathbf{a}, \mathbf{b})} \log p(y|\mathbf{a},\mathbf{b}; \theta_c) = \mathbb{E}_{c \sim q(c|y, \mathbf{a}, \mathbf{b})} \left [\nabla \log p(y|\mathbf{a},\mathbf{b}; \theta_c) + \log p(y|\mathbf{a},\mathbf{b}; \theta_c)  \nabla \log q(c|y, \mathbf{a}, \mathbf{b})\right]
$$


At inference time, to get $p(y|\mathbf{a}, \mathbf{b}; \theta)$ we use enumeration to calculate it exactly. For posterior inference, we can either use $q(c| y, \mathbf{a}, \mathbf{b})$ to approximate the true posterior or use Bayes rule to calculate the posterior exactly.

To interpret what specialized knowledge each component $c$ learns, we can find those examples whose posterior reaches maximum at $c$. 

When a model is trained, use the following test function to produce predictions, and then upload your best result to the kaggle competition:  https://www.kaggle.com/c/harvard-cs287-s19-hw4

In [0]:
def test_code(model):
    "All models should be able to be run with following command."
    upload = []
    # Update: for kaggle the bucket iterator needs to have batch_size 10
    test_iter = torchtext.data.BucketIterator(test, train=False, batch_size=10)
    for batch in test_iter:
        # Your prediction data here (don't cheat!)
        probs = model(batch.text)
        # here we assume that the name for dimension classes is `classes`
        _, argmax = probs.max('classes')
        upload += argmax.tolist()

    with open("predictions.txt", "w") as f:
        for u in upload:
            f.write(str(u) + "\n")

In addition, you should put up a (short) write-up following the template provided in the repository:  https://github.com/harvard-ml-courses/nlp-template